In [ ]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def read_trajectory(filename):
    traj = []
    with open(filename, "r") as f:
        metastr = f.readline()
        while metastr:
            metadata = list(map(int, metastr.split()))
            mat = np.zeros(shape=(4, 4))
            for i in range(4):
                matstr = f.readline()
                mat[i, :] = np.fromstring(matstr, dtype=float, sep=" \t")
            # traj.append(CameraPose(metadata, mat))
            traj.append(mat)
            metastr = f.readline()
    return traj

In [ ]:
def prepare_dataset():
    redwood = o3d.data.SampleRedwoodRGBDImages()
    camera_poses = read_trajectory(redwood.odometry_log_path)
    return redwood, camera_poses


redwood, camera_poses = prepare_dataset()

In [ ]:
fig = plt.figure(figsize=(12, len(camera_poses) * 4))
for i in range(len(camera_poses)):
    color = o3d.io.read_image(redwood.color_paths[i])
    depth = o3d.io.read_image(redwood.depth_paths[i])
    ax1 = fig.add_subplot(len(camera_poses), 2, 1 + 2 * i)
    ax1.imshow(np.asarray(color))
    ax1.axis("off")
    ax2 = fig.add_subplot(len(camera_poses), 2, 2 + 2 * i)
    ax2.imshow(np.asarray(depth))
    ax2.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
volume = o3d.pipelines.integration.ScalableTSDFVolume(
    voxel_length=4.0 / 512.0,
    sdf_trunc=0.04,
    color_type=o3d.pipelines.integration.TSDFVolumeColorType.RGB8,
)

for i in range(len(camera_poses)):
    print("Integrate {:d}-th image into the volume.".format(i))
    color = o3d.io.read_image(redwood.color_paths[i])
    depth = o3d.io.read_image(redwood.depth_paths[i])
    rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(
        color, depth, depth_trunc=4.0, convert_rgb_to_intensity=False
    )
    volume.integrate(
        rgbd,
        o3d.camera.PinholeCameraIntrinsic(
            o3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault
        ),
        np.linalg.inv(camera_poses[i]),
    )

In [ ]:
print("Extract a triangle mesh from the volume and visualize it.")
mesh = volume.extract_triangle_mesh()
mesh.compute_vertex_normals()
o3d.visualization.draw_geometries(
    [mesh],
    front=[0.5297, -0.1873, -0.8272],
    lookat=[2.0712, 2.0312, 1.7251],
    up=[-0.0558, -0.9809, 0.1864],
    zoom=0.47,
)